In [16]:
# Clonar o repositório oficial
!git clone https://github.com/mediar-ai/screenpipe.git

# Instalar bibliotecas para o Llama
!pip install -q transformers accelerate bitsandbytes huggingface_hub

fatal: destination path 'screenpipe' already exists and is not an empty directory.


In [ ]:
import os
from huggingface_hub import login
from transformers import pipeline
import torch

# Autentique com seu Token
from google.colab import userdata
try:
    # No Colab, voce pode adicionar o próprio token na aba 'Secrets' (ícone de chave)
    login(token=userdata.get('HF_TOKEN'))
except:
    # Alternativa manual para ambiente local
    from getpass import getpass
    login(getpass("Digite seu Hugging Face Token: "))

# Carregar o Llama 3.1 8B Instruct
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True},
    device_map="auto",
)

# FUNÇÃO PARA LER OS ARQUIVOS CLONADOS
def ler_arquivo_real(caminho_relativo):
    caminho_abs = os.path.join("/content/screenpipe", caminho_relativo)
    if os.path.exists(caminho_abs):
        with open(caminho_abs, 'r', encoding='utf-8') as f:
            return f.read()[:2500] # Limite para caber no contexto
    return f"ERRO: Arquivo {caminho_relativo} não encontrado no caminho {caminho_abs}"

# Extraindo os dados do repositório clonado
conteudo_ci = ler_arquivo_real(".github/workflows/ci.yml")
conteudo_contributing = ler_arquivo_real("CONTRIBUTING.md")

# Listar os changelogs para provar a frequência (estratégia de release)
pasta_changelogs = "/content/screenpipe/content/changelogs"
if os.path.exists(pasta_changelogs):
    arquivos_log = sorted(os.listdir(pasta_changelogs))[-15:] # Últimos 15 arquivos
    lista_changelogs = "\n".join(arquivos_log)
else:
    lista_changelogs = "Pasta de changelogs não encontrada."

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [19]:
prompt = f"""
Você é um especialista em Engenharia de Software. Analise os artefatos reais do projeto Screenpipe:

1. ARQUIVO DE CI (GitHub Actions):
{conteudo_ci}

2. GUIA DE CONTRIBUIÇÃO (Branching Model):
{conteudo_contributing}

3. LISTA DE ÚLTIMOS CHANGELOGS (Frequência de Release):
{lista_changelogs}

Responda em Português do Brasil:
- Identifique a Estratégia de Releases (Rapid Releases, Release Train ou LTS). Justifique pela frequência dos arquivos de changelog.
- Identifique o Modelo de Fluxo de Trabalho (GitHub Flow ou Gitflow). Justifique examinando se há branch 'develop' ou se o foco é na 'main'.
"""

messages = [{"role": "user", "content": prompt}]
output = pipe(messages, max_new_tokens=1000)
print(output[0]['generated_text'][-1]['content'])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Estratégia de Releases**

A partir da lista de últimos changelogs, podemos observar que há um grande número de releases, com frequência de aproximadamente um release por semana. Isso indica que a estratégia de releases é de tipo **Rapid Releases**. A frequência dos arquivos de changelog é um indicador de que a equipe está trabalhando em um ciclo de desenvolvimento contínuo, com releases frequentes e atualizações regulares.

**Justificativa**

A estratégia de rapid releases é adequada para projetos que:

* Requerem atualizações constantes para manter-se competitivos
* Tem uma base de usuários ativos e expectativas de atualizações regulares
* Possuem uma equipe de desenvolvimento ativa e capaz de gerenciar o fluxo de trabalho de forma eficiente

**Modelo de Fluxo de Trabalho**

O modelo de fluxo de trabalho é baseado no **GitHub Flow**. Isso pode ser inferido pelo fato de que:

* Não há uma branch 'develop' explícita, o que é um indicador de que a equipe está trabalhando diretamente na